In [2]:
import src.main as lc
import torch, os

# Filepaths
HHD = "/volumes/Ultra Touch"
MLP_LOC = HHD + "/MLP/LC_test"
COMPRESSED_SAVELOC = HHD + "/MLP/Compressed"
DECOMPRESSED_SAVELOC = HHD + "/MLP/Decompressed_test"

# Base dictionary (for decompressor to understand structure of the model)
BASE_DICT = torch.load(MLP_LOC + "/frame0.tar", map_location = torch.device('cpu'))["network_fn_state_dict"]

# 1. Compression of MLP model set

In [28]:
lc.compress_set(MLP_LOC, COMPRESSED_SAVELOC)

Loading: /volumes/Ultra Touch/MLP/LC_test/frame0.tar
Delta Compression on: frame1.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame1.tar
Delta Compression on: frame2.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame2.tar
Delta Compression on: frame3.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame3.tar
Delta Compression on: frame4.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame4.tar
Saving Compressed Format: frame0.tar
Saving Compressed Format: frame1.tar
Saving Compressed Format: frame2.tar
Saving Compressed Format: frame3.tar
Saving Compressed Format: frame4.tar


# 2. Decompression from Compressed Set

In [29]:
lc.load_compressed_set(COMPRESSED_SAVELOC, DECOMPRESSED_SAVELOC, BASE_DICT)

Decompressing for: compressed_frame0.pt
Decompressing for: compressed_frame1.pt
Decompressing for: compressed_frame2.pt
Decompressing for: compressed_frame3.pt
Decompressing for: compressed_frame4.pt
Saving Decompressed Model at: decompressed_frame0.pt
Saving Decompressed Model at: decompressed_frame1.pt
Saving Decompressed Model at: decompressed_frame2.pt
Saving Decompressed Model at: decompressed_frame3.pt
Saving Decompressed Model at: decompressed_frame4.pt


# 3. Testing Random Weights in decompressed MLP

In [30]:
frame_no = 3
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [31]:
original['pts_linears.1.weight'][2][10:20]

tensor([-0.1628, -0.0399,  0.0876, -0.0959,  0.2464, -0.0933, -0.0802,  0.0091,
         0.1737, -0.0667])

In [32]:
compressed['pts_linears.1.weight'][2][10:20]

tensor([-0.1436, -0.0426,  0.0437, -0.0939,  0.2938, -0.0950, -0.0698,  0.0255,
         0.1078, -0.0235])

In [33]:
original['pts_linears.2.weight'][4][10:20]

tensor([-0.0092,  0.0341, -0.3084, -0.0638,  0.0502,  0.1596, -0.0123,  0.0431,
        -0.0161,  0.1263])

In [34]:
compressed['pts_linears.2.weight'][4][10:20]

tensor([-0.0399,  0.0949, -0.3225, -0.1085,  0.0502,  0.1478, -0.0327,  0.0424,
        -0.0353,  0.1362])

In [36]:
frame_no = 1
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), 
                      map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [37]:
original['pts_linears.1.weight'][4][5:15]

tensor([-0.2480, -0.2322,  0.0630, -0.1238, -0.0873,  0.1134,  0.1539,  0.0085,
         0.0692, -0.0862])

In [38]:
compressed['pts_linears.1.weight'][4][5:15]

tensor([-0.1702, -0.2176,  0.0619, -0.1270, -0.0907,  0.1337,  0.1742,  0.0024,
         0.0702, -0.0826])

# 4. Compression Ratio

In [12]:
uncompressed_size = sum(os.path.getsize(MLP_LOC + "/" + f) for f in os.listdir(MLP_LOC))
compressed_size = sum(os.path.getsize(COMPRESSED_SAVELOC + "/" + f) for f in os.listdir(COMPRESSED_SAVELOC))

print("RESULTS")
print("Uncompressed MLP File Size (In MB): " + str(uncompressed_size / (10**6)))
print("Compressed MLP File Size (In MB): " + str(compressed_size / (10**6)))

# Compression Ratio = uncompressed / compressed

print("Compression Ratio (Tar VS LC-Checkpoint):")
print("{}%".format(round((uncompressed_size / compressed_size), 3) * 100))

# Space Savings = 1 - (compressed / uncompressed)

print("Space Savings (Tar VS LC-Checkpoint):")
print("{}%".format(round(1 - (compressed_size / uncompressed_size), 3) * 100))

RESULTS
Uncompressed MLP File Size (In MB): 71.781115
Compressed MLP File Size (In MB): 4.133173
Compression Ratio (Tar VS LC-Checkpoint):
1736.7%
Space Savings (Tar VS LC-Checkpoint):
94.19999999999999%


# 5. Save the torch dicts

In [40]:
for i in range(0, 5):
    compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(i))
    torch.save(compressed, HHD + "/MLP/test/frame_{}.pt".format(i))

In [41]:
for i in range(0, 5):
    print("Test original:")
    print(torch.load(HHD + "/MLP/test/frame_{}.pt".format(i))['pts_linears.1.weight'][2][10:15])
    print("Test restored:")
    print(torch.load(MLP_LOC + "/frame{}.tar".format(i), 
                     map_location = torch.device('cpu'))["network_fn_state_dict"]['pts_linears.1.weight'][2][10:15])

Test original:
tensor([-0.0529,  0.0227, -0.0604, -0.0509, -0.0415])
Test restored:
tensor([-0.0529,  0.0227, -0.0604, -0.0509, -0.0415])
Test original:
tensor([-0.1419, -0.1430, -0.0833, -0.1399, -0.0644])
Test restored:
tensor([-0.1713, -0.1642, -0.0814, -0.1256, -0.0692])
Test original:
tensor([-0.0531, -0.0542, -0.1290, -0.1169, -0.0189])
Test restored:
tensor([-0.0514, -0.0233, -0.1458, -0.1110, -0.0050])
Test original:
tensor([-0.1436, -0.0426,  0.0437, -0.0939,  0.2938])
Test restored:
tensor([-0.1628, -0.0399,  0.0876, -0.0959,  0.2464])
Test original:
tensor([-0.0529,  0.0033, -0.0472, -0.0968, -0.0209])
Test restored:
tensor([-0.0411, -0.0046, -0.0278, -0.0970, -0.0264])
